In [1]:
# Importar bibliotecas necessárias
import sys
from typing import Dict, List, Tuple, Set
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import numpy as np
from ipywidgets import interact, Dropdown, Button, Output, HBox, VBox, Label
from IPython.display import display, HTML

# Configurar matplotlib para melhor qualidade
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 9

## Implementação do Algoritmo de Dijkstra

In [2]:
class GrafoDijkstra:
    """Implementação do grafo com algoritmo de Dijkstra"""
    
    def __init__(self):
        self.grafo: Dict[str, Dict[str, float]] = {}
    
    def adicionar_vertice(self, vertice: str) -> None:
        if vertice not in self.grafo:
            self.grafo[vertice] = {}
    
    def adicionar_aresta(self, origem: str, destino: str, peso: float) -> None:
        if origem not in self.grafo:
            self.adicionar_vertice(origem)
        if destino not in self.grafo:
            self.adicionar_vertice(destino)
        
        self.grafo[origem][destino] = peso
        self.grafo[destino][origem] = peso
    
    def dijkstra(self, inicio: str, fim: str) -> Tuple[float, List[str]]:
        """Implementa o algoritmo de Dijkstra"""
        if inicio not in self.grafo or fim not in self.grafo:
            return float('inf'), []
        
        if inicio == fim:
            return 0, [inicio]
        
        # Inicializar estruturas
        distancias: Dict[str, float] = {}
        visitados: Set[str] = set()
        antecessores: Dict[str, str] = {}
        nao_visitados: List[str] = []
        
        for vertice in self.grafo:
            distancias[vertice] = float('inf')
            antecessores[vertice] = None
            nao_visitados.append(vertice)
        
        distancias[inicio] = 0
        
        # Algoritmo principal
        while nao_visitados:
            vertice_atual = None
            menor_distancia = float('inf')
            
            for vertice in nao_visitados:
                if distancias[vertice] < menor_distancia:
                    menor_distancia = distancias[vertice]
                    vertice_atual = vertice
            
            if vertice_atual is None or distancias[vertice_atual] == float('inf'):
                break
            
            visitados.add(vertice_atual)
            nao_visitados.remove(vertice_atual)
            
            if vertice_atual == fim:
                break
            
            for vizinho, peso in self.grafo[vertice_atual].items():
                if vizinho not in visitados:
                    nova_distancia = distancias[vertice_atual] + peso
                    if nova_distancia < distancias[vizinho]:
                        distancias[vizinho] = nova_distancia
                        antecessores[vizinho] = vertice_atual
        
        # Reconstruir caminho
        caminho = []
        vertice_atual = fim
        
        if antecessores[vertice_atual] is not None or vertice_atual == inicio:
            while vertice_atual is not None:
                caminho.insert(0, vertice_atual)
                vertice_atual = antecessores[vertice_atual]
        
        return distancias[fim], caminho
    
    def obter_vertices(self) -> List[str]:
        return sorted(list(self.grafo.keys()))

## Base de Dados - Capitais Brasileiras e Distâncias

In [3]:
# Capitais brasileiras com coordenadas
CAPITAIS_BRASIL = {
    "Brasília": {"lat": -15.7975, "lon": -47.8919, "estado": "DF"},
    "Rio Branco": {"lat": -9.9754, "lon": -67.8049, "estado": "AC"},
    "Maceió": {"lat": -9.6412, "lon": -35.7353, "estado": "AL"},
    "Macapá": {"lat": 0.3551, "lon": -51.0894, "estado": "AP"},
    "Manaus": {"lat": -3.1190, "lon": -60.0217, "estado": "AM"},
    "Salvador": {"lat": -12.9822, "lon": -38.5104, "estado": "BA"},
    "Fortaleza": {"lat": -3.7319, "lon": -38.5267, "estado": "CE"},
    "Vitória": {"lat": -20.2976, "lon": -40.2958, "estado": "ES"},
    "Goiânia": {"lat": -15.7942, "lon": -48.8758, "estado": "GO"},
    "São Luís": {"lat": -2.5352, "lon": -44.3068, "estado": "MA"},
    "Cuiabá": {"lat": -15.5880, "lon": -56.0884, "estado": "MT"},
    "Campo Grande": {"lat": -20.4697, "lon": -54.6201, "estado": "MS"},
    "Belo Horizonte": {"lat": -19.8267, "lon": -43.9449, "estado": "MG"},
    "Belém": {"lat": -1.4554, "lon": -48.5044, "estado": "PA"},
    "João Pessoa": {"lat": -7.1219, "lon": -34.8450, "estado": "PB"},
    "Curitiba": {"lat": -25.4284, "lon": -49.2733, "estado": "PR"},
    "Recife": {"lat": -8.0726, "lon": -34.8759, "estado": "PE"},
    "Rio de Janeiro": {"lat": -22.9068, "lon": -43.1729, "estado": "RJ"},
    "Natal": {"lat": -5.7945, "lon": -35.2110, "estado": "RN"},
    "Porto Alegre": {"lat": -30.0324, "lon": -51.2304, "estado": "RS"},
    "Boa Vista": {"lat": 2.8235, "lon": -60.6692, "estado": "RR"},
    "São Paulo": {"lat": -23.5505, "lon": -46.6333, "estado": "SP"},
    "Aracaju": {"lat": -10.9111, "lon": -37.0734, "estado": "SE"},
    "Palmas": {"lat": -10.2105, "lon": -48.3281, "estado": "TO"},
}

# Distâncias entre capitais (em Km)
DISTANCIAS = {
    ("Brasília", "Goiânia"): 207,
    ("Brasília", "Belo Horizonte"): 716,
    ("Brasília", "São Paulo"): 1004,
    ("Brasília", "Rio de Janeiro"): 1145,
    ("Brasília", "Cuiabá"): 870,
    ("Brasília", "Palmas"): 676,
    ("Rio Branco", "Manaus"): 1434,
    ("Rio Branco", "Boa Vista"): 1876,
    ("Maceió", "Recife"): 257,
    ("Maceió", "Salvador"): 486,
    ("Maceió", "Aracaju"): 285,
    ("Macapá", "Belém"): 630,
    ("Macapá", "Boa Vista"): 1112,
    ("Manaus", "Boa Vista"): 815,
    ("Manaus", "Belém"): 1468,
    ("Salvador", "Aracaju"): 277,
    ("Salvador", "Fortaleza"): 844,
    ("Salvador", "Belo Horizonte"): 1384,
    ("Salvador", "Rio de Janeiro"): 1751,
    ("Fortaleza", "Natal"): 494,
    ("Fortaleza", "São Luís"): 729,
    ("Fortaleza", "João Pessoa"): 631,
    ("Vitória", "Rio de Janeiro"): 525,
    ("Vitória", "Belo Horizonte"): 525,
    ("Vitória", "São Paulo"): 842,
    ("Goiânia", "Brasília"): 207,
    ("Goiânia", "Cuiabá"): 928,
    ("Goiânia", "Belo Horizonte"): 727,
    ("Goiânia", "São Paulo"): 924,
    ("São Luís", "Palmas"): 914,
    ("São Luís", "Belém"): 1001,
    ("São Luís", "Fortaleza"): 729,
    ("Cuiabá", "Campo Grande"): 808,
    ("Cuiabá", "Brasília"): 870,
    ("Cuiabá", "Goiânia"): 928,
    ("Campo Grande", "Cuiabá"): 808,
    ("Campo Grande", "São Paulo"): 1270,
    ("Campo Grande", "Curitiba"): 1373,
    ("Belo Horizonte", "Rio de Janeiro"): 434,
    ("Belo Horizonte", "São Paulo"): 588,
    ("Belo Horizonte", "Brasília"): 716,
    ("Belém", "Macapá"): 630,
    ("Belém", "São Luís"): 1001,
    ("Belém", "Manaus"): 1468,
    ("João Pessoa", "Recife"): 120,
    ("João Pessoa", "Natal"): 185,
    ("João Pessoa", "Fortaleza"): 631,
    ("Curitiba", "São Paulo"): 408,
    ("Curitiba", "Porto Alegre"): 710,
    ("Curitiba", "Campo Grande"): 1373,
    ("Recife", "Maceió"): 257,
    ("Recife", "João Pessoa"): 120,
    ("Recife", "Salvador"): 802,
    ("Rio de Janeiro", "São Paulo"): 429,
    ("Rio de Janeiro", "Belo Horizonte"): 434,
    ("Rio de Janeiro", "Vitória"): 525,
    ("Natal", "Fortaleza"): 494,
    ("Natal", "João Pessoa"): 185,
    ("Porto Alegre", "Curitiba"): 710,
    ("Porto Alegre", "São Paulo"): 1139,
    ("Boa Vista", "Macapá"): 1112,
    ("Boa Vista", "Manaus"): 815,
    ("São Paulo", "Rio de Janeiro"): 429,
    ("São Paulo", "Belo Horizonte"): 588,
    ("São Paulo", "Curitiba"): 408,
    ("São Paulo", "Brasília"): 1004,
    ("Aracaju", "Maceió"): 285,
    ("Aracaju", "Salvador"): 277,
    ("Palmas", "Brasília"): 676,
    ("Palmas", "São Luís"): 914,
}

print(f"Total de capitais: {len(CAPITAIS_BRASIL)}")
print(f"Total de conexões: {len(DISTANCIAS)}")

Total de capitais: 24
Total de conexões: 70


## Inicializar Grafo

In [4]:
# Construir grafo
grafo = GrafoDijkstra()

for capital in CAPITAIS_BRASIL.keys():
    grafo.adicionar_vertice(capital)

for (cidade1, cidade2), distancia in DISTANCIAS.items():
    grafo.adicionar_aresta(cidade1, cidade2, distancia)

print("Grafo construído com sucesso!")
print(f"Vértices: {len(grafo.grafo)}")
print(f"Arestas: {sum(len(neighbors) for neighbors in grafo.grafo.values()) // 2}")

Grafo construído com sucesso!
Vértices: 24
Arestas: 42


## Preparar Coordenadas para Visualização

In [5]:
# Calcular coordenadas normalizadas
def calcular_coordenadas():
    lats = [info["lat"] for info in CAPITAIS_BRASIL.values()]
    lons = [info["lon"] for info in CAPITAIS_BRASIL.values()]
    
    min_lat, max_lat = min(lats), max(lats)
    min_lon, max_lon = min(lons), max(lons)
    
    coords = {}
    for capital, info in CAPITAIS_BRASIL.items():
        x = (info["lon"] - min_lon) / (max_lon - min_lon)
        y = (info["lat"] - min_lat) / (max_lat - min_lat)
        coords[capital] = (x, y)
    
    return coords

coords_capitais = calcular_coordenadas()
print("Coordenadas calculadas!")
print(f"Primeira capital: {list(coords_capitais.items())[0]}")

Coordenadas calculadas!
Primeira capital: ('Brasília', (0.6041583864028715, 0.4332524752023229))


## Função de Visualização

In [6]:
def visualizar_caminho(origem, destino):
    """Visualiza o caminho mais curto entre duas cidades"""
    
    # Calcular caminho
    distancia, caminho = grafo.dijkstra(origem, destino)
    
    # Criar figura
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # Desenhar arestas
    for (cidade1, cidade2), dist in DISTANCIAS.items():
        x1, y1 = coords_capitais[cidade1]
        x2, y2 = coords_capitais[cidade2]
        
        # Verificar se está no caminho
        em_caminho = False
        if len(caminho) > 1:
            for i in range(len(caminho) - 1):
                c1, c2 = caminho[i], caminho[i + 1]
                if (c1 == cidade1 and c2 == cidade2) or (c1 == cidade2 and c2 == cidade1):
                    em_caminho = True
                    break
        
        if em_caminho:
            ax.plot([x1, x2], [y1, y2], 'r-', linewidth=4, zorder=2)
        else:
            ax.plot([x1, x2], [y1, y2], 'lightgray', linewidth=0.5, linestyle='--', zorder=0)
    
    # Desenhar vértices com espaçamento melhorado para labels
    for capital, (x, y) in coords_capitais.items():
        if capital == origem:
            cor = 'green'
            tamanho = 250
            marcador = 'o'
            zorder = 5
        elif capital == destino:
            cor = 'red'
            tamanho = 250
            marcador = 's'
            zorder = 5
        elif capital in caminho:
            cor = 'gold'
            tamanho = 120
            marcador = 'o'
            zorder = 4
        else:
            cor = 'lightblue'
            tamanho = 60
            marcador = 'o'
            zorder = 2
        
        ax.scatter(x, y, s=tamanho, c=cor, marker=marcador, 
                  edgecolors='black', linewidth=1.5, zorder=zorder)
    
    # Adicionar labels com offset para evitar sobreposição
    for capital, (x, y) in coords_capitais.items():
        if capital == origem:
            cor_texto = 'darkgreen'
            peso = 'bold'
            size = 8
            offset_y = -0.12
        elif capital == destino:
            cor_texto = 'darkred'
            peso = 'bold'
            size = 8
            offset_y = -0.12
        elif capital in caminho:
            cor_texto = 'darkorange'
            peso = 'bold'
            size = 7
            offset_y = -0.10
        else:
            cor_texto = 'black'
            peso = 'normal'
            size = 6.5
            offset_y = -0.08
        
        ax.text(x, y + offset_y, capital, fontsize=size, ha='center', va='top',
               color=cor_texto, weight=peso, zorder=6,
               bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7, edgecolor='none'))
    
    # Adicionar distâncias nas arestas do caminho
    if len(caminho) > 1:
        for i in range(len(caminho) - 1):
            c1, c2 = caminho[i], caminho[i + 1]
            x1, y1 = coords_capitais[c1]
            x2, y2 = coords_capitais[c2]
            
            # Encontrar distância
            chave1 = (c1, c2)
            chave2 = (c2, c1)
            dist = DISTANCIAS.get(chave1) or DISTANCIAS.get(chave2)
            
            mx, my = (x1 + x2) / 2, (y1 + y2) / 2
            ax.text(mx, my, f"{dist:.0f}km", fontsize=9, ha='center', va='center',
                   bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.9, edgecolor='red', linewidth=1),
                   weight='bold', color='darkred', zorder=7)
    
    # Configurar eixos
    ax.set_xlim(-0.15, 1.15)
    ax.set_ylim(-0.15, 1.15)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.invert_yaxis()
    
    # Título e informações
    if distancia == float('inf'):
        titulo = f"Não há caminho de {origem} para {destino}"
        resultado = "SEM ROTA"
    else:
        caminho_str = " → ".join(caminho)
        titulo = f"Caminho: {caminho_str}\nDistância Total: {distancia:.0f} km"
        resultado = f"✓ {distancia:.0f} km"
    
    ax.set_title(titulo, fontsize=14, weight='bold', pad=20)
    
    # Adicionar legenda
    verde = mpatches.Patch(color='green', label='Origem')
    vermelho = mpatches.Patch(color='red', label='Destino')
    ouro = mpatches.Patch(color='gold', label='Caminho')
    azul = mpatches.Patch(color='lightblue', label='Outras cidades')
    ax.legend(handles=[verde, vermelho, ouro, azul], loc='upper left', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Exibir resultado
    print(f"\n{'='*60}")
    print(f"ORIGEM: {origem}")
    print(f"DESTINO: {destino}")
    print(f"DISTÂNCIA MÍNIMA: {resultado}")
    if distancia != float('inf'):
        print(f"CAMINHO: {caminho_str}")
    print(f"{'='*60}\n")

print("Função de visualização pronta!")

Função de visualização pronta!


## Interface Interativa

In [7]:
# Criar dropdowns
capitais_ordenadas = sorted(list(CAPITAIS_BRASIL.keys()))

dropdown_origem = Dropdown(
    options=capitais_ordenadas,
    value='São Paulo',
    description='Origem:',
    style={'description_width': '100px'}
)

dropdown_destino = Dropdown(
    options=capitais_ordenadas,
    value='Rio de Janeiro',
    description='Destino:',
    style={'description_width': '100px'}
)

# Criar output para resultado
output = Output()

# Função para botão
def ao_clicar_calcular(b):
    output.clear_output(wait=True)
    with output:
        if dropdown_origem.value == dropdown_destino.value:
            print("⚠️ Origem e destino devem ser diferentes!")
        else:
            visualizar_caminho(dropdown_origem.value, dropdown_destino.value)

# Criar botão
btn_calcular = Button(
    description='Calcular Caminho',
    button_style='success',
    tooltip='Calcula o caminho mais curto',
    icon='play'
)
btn_calcular.on_click(ao_clicar_calcular)

# Organizar interface
titulo = Label(value="Escolha a origem e o destino:")
box_selecao = HBox([dropdown_origem, dropdown_destino, btn_calcular])
box_principal = VBox([titulo, box_selecao])

display(box_principal)
display(output)

Output()

## Exemplos de Uso

### Teste alguns caminhos:

1. **São Paulo → Rio de Janeiro** (Sudeste)
2. **Brasília → Manaus** (Centro-Norte)
3. **Porto Alegre → Boa Vista** (Sul-Norte)
4. **Recife → Salvador** (Nordeste)
5. **Curitiba → São Paulo** (Caminho curto)

Use a interface acima para explorar diferentes rotas!

## Informações sobre o Algoritmo

### Complexidade:
- **Tempo**: O(n²) onde n = número de vértices
- **Espaço**: O(n) para armazenar distâncias

### Como funciona:
1. Inicializa distâncias infinitas para todos os nós exceto a origem
2. Seleciona o nó com menor distância não visitado
3. Atualiza as distâncias dos vizinhos
4. Repete até alcançar o destino

### Características:
✓ Implementação 100% em Python puro (sem bibliotecas externas para o algoritmo)  
✓ Funciona com grafos não-direcionados  
✓ Garante encontrar o caminho mais curto  
✓ Eficiente para grafos pequenos e médios